In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [3]:
import os

In [4]:
import uuid

In [5]:
import pickle
import pandas as pd

In [5]:
from sklearn.feature_extraction import DictVectorizer

In [6]:
import mlflow

In [7]:
# import xgboost as xgb

In [8]:
# from google.cloud import storage

In [9]:
# def download_directory(bucket_name, directory_path, destination_path):
#     storage_client = storage.Client()
#     bucket = storage_client.bucket(bucket_name)

#     blobs = bucket.list_blobs(prefix=directory_path)

#     for blob in blobs:
#         # Extract the relative path from the blob's name
#         relative_path = os.path.relpath(blob.name, directory_path)

#         # Create the destination directory if it doesn't exist
#         dest_directory = os.path.join(destination_path, os.path.dirname(relative_path))
#         os.makedirs(dest_directory, exist_ok=True)

#         # Download the blob to the destination directory
#         destination_file = os.path.join(dest_directory, os.path.basename(relative_path))
#         blob.download_to_filename(destination_file)

#     print("Directory downloaded successfully.")



In [10]:
# bucket_name='mlops-zoomcamp'

In [11]:
# directory_path='2/45e0e8d445544eef87226b6cbbf1921f/artifacts/models_mlflow/'

In [12]:
# destination_path="/home/konradballegro/notebooks/inputs/45e0e8d445544eef87226b6cbbf1921f"

In [13]:
# download_directory(bucket_name, directory_path, destination_path)

In [14]:
# def download_model(bucket_name, artifact_path, destination_path):
#     storage_client = storage.Client()
#     bucket = storage_client.bucket(bucket_name)
#     blob = bucket.blob(artifact_path)
#     blob.download_to_filename(destination_path)

In [15]:
# download_model(
#         bucket_name='mlops-zoomcamp',
#         artifact_path='data/green_tripdata_2021-02.parquet',
#         destination_path="/home/konradballegro/notebooks/inputs/model.xgb"
#         )

In [16]:
with open('/home/konradballegro/notebooks/inputs/model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [17]:
dv

DictVectorizer()

In [18]:
model

LinearRegression()

In [19]:
categorical = ['PULocationID', 'DOLocationID']

In [20]:
def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [21]:
# df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-03.parquet')

In [22]:
# df.info()

In [23]:
RUN_ID='8e35278e4321423d8029f87ec106243f'

In [24]:
input_file='https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-04.parquet'

In [25]:
output_file='/home/konradballegro/notebooks/outputs/prediction.parquet'

In [26]:
def generate_uuids(n):
    ride_ids = []
    for i in range(n):
        ride_ids.append(str(uuid.uuid4()))
    return ride_ids

In [27]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    df['ride_id'] = generate_uuids(len(df))

    return df

In [28]:
def prepare_dictionaries(df: pd.DataFrame):
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']

    categorical = ['PU_DO']
    numerical = ['trip_distance']
#     dicts = df[categorical + numerical].to_dict(orient='records')
    dicts = df[categorical].to_dict(orient='records')
    dicts = dv.transform(dicts)
    
    return dicts

In [29]:
# def load_model(run_id):
#     model=mlflow.pyfunc.load_model('/home/konradballegro/notebooks/inputs/45e0e8d445544eef87226b6cbbf1921f/')
#     return model

In [30]:
def apply_model(input_file, run_id, output_file):

    df = read_dataframe(input_file)
    print('df read')
    dicts = prepare_dictionaries(df)
    print('dicts prepared')
    
#     model = load_model(run_id)
#     print('model loaded')
    y_pred = model.predict(dicts)
    print('predictions made')
    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['tpep_pickup_datetime'] = df['tpep_pickup_datetime']
    df_result['PULocationID'] = df['PULocationID']
    df_result['DOLocationID'] = df['DOLocationID']
    df_result['actual_duration'] = df['duration']
    df_result['predicted_duration'] = y_pred
    df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
    df_result['model_version'] = run_id
    
    df_result.to_parquet(output_file, index=False)
    print('predictions saved')

In [31]:
apply_model(input_file=input_file, run_id=RUN_ID, output_file=output_file)

df read
dicts prepared
predictions made
predictions saved


In [32]:
predictions = pd.read_parquet(output_file)

In [33]:
predictions

,ride_id,tpep_pickup_datetime,PULocationID,DOLocationID,actual_duration,predicted_duration,diff,model_version
0,6e2d1652-e05d-4eda-b552-83edb6557bea,2022-04-01 00:21:13,163,62,37.333333,24.781802,12.551531,8e35278e4321423d8029f87ec106243f
1,8befd736-9a6c-4790-8a63-332499bfd9a7,2022-04-01 00:07:47,142,141,11.416667,24.781802,-13.365136,8e35278e4321423d8029f87ec106243f
2,fadcdedf-0675-4848-8e67-a9e5cd67fb2f,2022-04-01 00:14:52,79,148,8.850000,24.781802,-15.931802,8e35278e4321423d8029f87ec106243f
3,8a221402-ef4c-4638-b006-c36d1169034d,2022-04-01 00:30:02,79,13,15.066667,24.781802,-9.715136,8e35278e4321423d8029f87ec106243f
4,b1f6d499-f0f4-4809-ba24-bd4f827dacb6,2022-04-01 00:48:40,238,116,14.900000,24.781802,-9.881802,8e35278e4321423d8029f87ec106243f
...,...,...,...,...,...,...,...,...
3501815,7ec3063c-738f-4c77-b301-cef8cec68be3,2022-04-30 23:10:00,80,82,18.000000,24.781802,-6.781802,8e35278e4321423d8029f87ec106243f
3501816,d185439c-6a65-495e-9022-b1629e0769a7,2022-04-30 23:28:59,48,264,17.650000,24.781802,-7.131802,8e35278e4321423d8029f87ec106243f
3501817,5d653ec1-7217-4d02-9f5c-3ea06f9b07f4,2022-04-30 23:27:25,137,143,17.583333,24.781802,-7.198469,8e35278e4321423d8029f87ec106243f
3501818,f6b657f5-dcd3-430b-a3f1-0708d606b712,2022-04-30 23:24:26,141,112,19.316667,24.781802,-5.465136,8e35278e4321423d8029f87ec106243f


In [34]:
duration_std = predictions['predicted_duration'].std()

print("Standard deviation of trip duration:", duration_std)

Standard deviation of trip duration: 1.882938518615699e-13


In [35]:
duration_mean = predictions['predicted_duration'].mean()

print("Mean of trip duration:", duration_mean)

Mean of trip duration: 24.7818021688019
